# Definitions and imports

## Imports

In [ ]:
# 
# Generic; file access
# import os as os
# from __future__ import division
# 
# Computational libs
import numpy as np
# from scipy.linalg import norm
# from scipy import linspace
#
# Pycuda
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda.gpuarray import to_gpu
# 
# Minimizers
# import nlopt
from ipopt import minimize_ipopt
from scipy.optimize import minimize
#
# Graphing utilities
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# from autograd import grad
# import autograd.numpy as np
# from numba import jit, float64, uint8, prange

---

## Plotting routine

In [ ]:
def pplot(x,dim):
    X3 = x.reshape((-1,dim))
    %matplotlib notebook
    r = 1
    coeff = .94
    # phi, theta = nmp.mgrid[0.0:nmp.pi:50j, 0.0:2.0*nmp.pi:50j]
    # x = r*nmp.sin(phi)*nmp.cos(theta)
    # y = r*nmp.sin(phi)*nmp.sin(theta)
    # z = r*nmp.cos(phi)
    #Set colors and render
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # ax.plot_wireframe(coeff*x, coeff*y, coeff*z,  rstride=4, cstride=4,  color="blue", alpha=0.3,linewidth=1)
    # ax.plot_surface(coeff*x, coeff*y, coeff*z,  rstride=4, cstride=4, color='lightgray', alpha=0.9, linewidth=.3)

    ax.scatter(X3[:,0],X3[:,1],X3[:,2], marker='o', color='red')

    ax.set_xlim([0,1])
    ax.set_ylim([0,1])
    ax.set_zlim([0,1])
    ax.set_aspect("equal")
    plt.tight_layout()
    plt.show()

## Define energy and verify gradient

$$ E(X) = \sum_{i,j} e^{-C\| x_i - x_j\|^2} = \sum_{i,j} \exp\left(- C\left[ (x_i^1 - x_j^1)^2 + (x_i^2 - x_j^2)^2 + (x_i^3 - x_j^3)^2 \right]\right) $$
$$ \frac{\partial E}{\partial x_i^l}  = \sum_{j} -2 C(x_i^l - x_j^l) \cdot \exp\left(-C\| x_i - x_j\|^2\right)  $$

In [ ]:
def gaussian(X, grad, C, dim):
        en_all = 0;
        for l in range(dim):
            en_all = en_all - C*(X.reshape((-1, dim))[:,l][None,:] - X.reshape((-1, dim))[:,l][:,None])**2.
        en_all = np.exp(en_all)
        for l in range(dim):
            grad.reshape((-1, dim))[:,l] = -C * 2. * np.sum(
                en_all*(-2*(
                X.reshape((-1, dim))[:,l][None,:] - X.reshape((-1, dim))[:,l][:,None]
            ))
                                                  , 1)
        return en_all.sum()

def sph2cart(phitheta):
    return np.array([cos(phitheta[0])*sin(phitheta[1]), sin(phitheta[0])*sin(phitheta[1]), cos(phitheta[1])])


In [ ]:
# SciPy is somewhat different in terms of function/gradient calls
def gaussian_scp(X, C, dim):
    en_all = 0;
    for l in range(dim):
        en_all = en_all - C*(X.reshape((-1, dim))[:,l][None,:] - X.reshape((-1, dim))[:,l][:,None])**2.
    en_all = np.exp(en_all)
    return en_all.sum()
def gaussian_scp_grad(X, C, dim):
    grad = np.zeros_like(X)
    en_all = 0;
    for l in range(dim):
        en_all = en_all - C*(X.reshape((-1, dim))[:,l][None,:] - X.reshape((-1, dim))[:,l][:,None])**2.
    en_all = np.exp(en_all)
    for l in range(dim):
        grad.reshape((-1, dim))[:,l] = C * 2. * np.sum(
            en_all*(-2*(
            X.reshape((-1, dim))[:,l][None,:] - X.reshape((-1, dim))[:,l][:,None]
        ))
                                              , 1)
    return -grad

scipy.optimize.check_grad returns the **error magnitude**

In [ ]:
from scipy.optimize import check_grad

In [ ]:
C = 2.
dim =3

In [ ]:
def ffunc(x):
    return gaussian(x,np.zeros_like(x), C, dim)
def fgrad(x):
    v = np.zeros_like(x) 
    gaussian(x,v, C, dim)
    return v
    
u = np.random.randn(30)
(check_grad(lambda X: gaussian_scp(X, C, dim),lambda X:  gaussian_scp_grad(X,C,dim), u),
check_grad(ffunc,fgrad, u))

---

# PyCuda

In [ ]:
modE = SourceModule("""
#define DIM 3
#define BLOCK_SIZE 256 
//typedef struct { double x, y, z, e; } Body;

__global__                                                           
void energy(double *pt, double3 *p, double* c, int n) {                          
    const int i = blockDim.x * blockIdx.x + threadIdx.x;
     if (i < n) {
        double S = 0.0;

        for (int j = 0; j < n; j++) {
            double dx = p[j].x - p[i].x;
            double dy = p[j].y - p[i].y;
            double dz = p[j].z - p[i].z;
            double distSqr = dx*dx + dy*dy + dz*dz;

            S += exp(- *c *distSqr);
        }
        pt[i] = S;
    }
}                                                                  
""")

modG = SourceModule("""
#define DIM 3

//typedef struct { double x, y, z, e;  } Body;

__global__                                                           
void gradient(double3* grad, double3 *p, double* c, int n) {                          
    const int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) {
    double Sx = 0.0;
    double Sy = 0.0;
    double Sz = 0.0;

    for (int j = 0; j < n; j++) {
      double dx = p[j].x - p[i].x;
      double dy = p[j].y - p[i].y;
      double dz = p[j].z - p[i].z;
      double S = exp(- *c *(dx*dx + dy*dy + dz*dz));

      Sx += 4.0* *c *dx*S;
      Sy += 4.0* *c *dy*S;
      Sz += 4.0* *c *dz*S;
    }
    
    grad[i].x = Sx;
    grad[i].y = Sy;
    grad[i].z = Sz;
  }
}                                                                  
""")
energy = modE.get_function("energy")
gradient = modG.get_function("gradient")

In [ ]:
c = 30
n = 10000
dim = 3

In [ ]:
c = c * np.ones(1, dtype='float64')
n = n*np.ones(1,dtype='uint32')
c_dev = to_gpu(c)
n_dev = to_gpu(n)

cnf = np.random.random(dim*n)
pt = np.zeros(n, dtype='float64')
grad = np.zeros(dim*n, dtype='float64')

cnf_dev = to_gpu(cnf)
grad_dev = to_gpu(grad)
pt_dev = to_gpu(pt)

In [ ]:
def gaussian_gpu(cnf, pt_dev, cnf_dev, c_dev, n):
    blocksize = 256
    numblocks = int(((n + blocksize - 1) // blocksize)[0])
    cnf_dev.set(cnf)
    energy(pt_dev, cnf_dev, c_dev, n, block=(blocksize,1,1), grid=(numblocks,1))
    pt_dev.get(pt)
    return pt.sum()

def gaussian_gpu_grad(cnf, grad_dev, cnf_dev, c_dev, n):
    blocksize = 256
    numblocks = int(((n + blocksize - 1) // blocksize)[0])
    cnf_dev.set(cnf)
    gradient(grad_dev, cnf_dev, c_dev, n, block=(blocksize,1,1), grid=(numblocks,1))
#     pt_dev.get(pt)
    return grad_dev.get()

In [ ]:
(
    np.abs(ens.sum()-gaussian_scp(cnf, 4.0, dim)),
    norm(grad - gaussian_scp_grad(cnf, 4.0, 3) )
)

# Optimization

## Initialize and define bounds

CPU:

In [ ]:
C = 8.
dim = 3
numvars = 1000
u = np.random.random((numvars,dim))
u = u.flatten()
# Bounds
lb = np.zeros_like(u)
ub = np.ones_like(u)
bnds = tuple([(lb[i],ub[i]) for i in range(numvars*dim) ])

---

Pycuda:

In [ ]:
# execute this cell to change parameters; 
c = 40
n = 10000
dim = 3

c = c * np.ones(1, dtype='float64')
n = n*np.ones(1,dtype='uint32')
c_dev = to_gpu(c)
n_dev = to_gpu(n)

cnf = np.random.random(dim*n)
pt = np.zeros(n, dtype='float64')
grad = np.zeros(dim*n, dtype='float64')

cnf_dev = to_gpu(cnf)
grad_dev = to_gpu(grad)
pt_dev = to_gpu(pt)

# Bounds
lb = np.zeros_like(cnf)
ub = np.ones_like(cnf)
bnds = tuple([(lb[i],ub[i]) for i in range(n[0]*dim) ])

## SciPy

CPU:

In [ ]:
scpres = minimize(lambda X: gaussian_scp(X, C, dim), u,
                  jac=lambda X: gaussian_scp_grad(X, C, dim), method='L-BFGS-B',
            bounds=bnds)
scpres.fun

Pycuda evaluation:

In [ ]:
scpres = minimize(lambda X: gaussian_gpu(X, pt_dev, cnf_dev, c_dev, n), 
             cnf, jac=lambda X: gaussian_gpu_grad(X, grad_dev, cnf_dev, c_dev, n), method='L-BFGS-B',
            bounds=bnds)
scpres.fun

## Ipopt

CPU:

In [ ]:
res = minimize_ipopt(lambda X: gaussian_scp(X, C, dim), u, jac=lambda X: gaussian_scp_grad(X, C, dim),
                     bounds=bnds, options={'maxiter': 1000}) 
res.fun, res.success

Pycuda evaluation:

In [ ]:
res = minimize(lambda X: gaussian_gpu(X, pt_dev, cnf_dev, c_dev, n), 
             cnf, jac=lambda X: gaussian_gpu_grad(X, grad_dev, cnf_dev, c_dev, n),
            bounds=bnds, options={'maxiter': 1000})
res.fun, res.success

---

## NLopt

In [ ]:
# Initialize the solver
opt = nlopt.opt(nlopt.LD_LBFGS, np.size(u))
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)
opt.set_min_objective(lambda x,v: gaussian(x,v, C, dim))
opt.set_ftol_rel(1e-10)
# Invoke NLopt
x = opt.optimize(u)
minf = opt.last_optimum_value()

---

In [ ]:
res

In [ ]:
pplot(res.x,dim)

## Clustering

In [ ]:
#Counts multiplicity of occurances in an array upto set precision, returns a list
def multiplicity_array(flat_vector_array,precision,dim):
    vector_array = flat_vector_array.reshape((-1,dim))
    hash_vector_array = map(tuple,np.round(vector_array,precision))
    counter = collections.Counter(hash_vector_array)
    return counter.most_common()


#initial test for 
#X3count = res.x.reshape((-1,dim))
precision=6
#X3counthash=map(str,np.round(X3count,precision))
#counter=collections.Counter(X3counthash)

#print(len(np.round(X3count,precision)),len(counter))
#counter

In [ ]:
%%timeit
m = multiplicity_array(res.x, 6 ,3)

In [ ]:
%%timeit
mm, nn = np.unique(np.round(res.x, precision).reshape((-1,dim)), axis=0, return_counts=True)

## Lattice initialization

In [ ]:
# Generators
genbcc = np.array([[1., 1., -1.],[-1., 1., 1.],[1., -1., 1.]])
genfcc = np.array([[1., 1., 0.],[0., 1., 1.],[1., 0., 1.]])

### Make bcc

In [ ]:
n = 100  # number of pts per side
A = 1.
a = b = c = np.linspace(0., A, n)
mesh = np.meshgrid(a,b,c)
zlattice = np.vstack((mesh[0].flatten(), mesh[1].flatten(), mesh[2].flatten())).T
del mesh

latticebcc = np.vstack((zlattice+ A/(2*(n-1)), zlattice))

inclusion = np.all(np.logical_and(latticebcc>=.0,  latticebcc <=1.), axis=1)
latticebcc = latticebcc[inclusion,:]
# pplot(latticebcc, 3)
# lattice.shape

In [ ]:
latticebcc.shape

### Make fcc

In [ ]:
n = 2  # number of pts per side
A = 1.
a = b = c = np.linspace(0., A, n)
mesh = np.meshgrid(a,b,c)
zlattice = np.vstack((mesh[0].flatten(), mesh[1].flatten(), mesh[2].flatten())).T
del mesh

genfcc = A*np.array([[1., 1., 0.],[0., 1., 1.],[1., 0., 1.]])/(2*(n-1))
latticefcc = np.vstack(( zlattice + genfcc[0,:], zlattice + genfcc[1,:], zlattice + genfcc[2,:] ))

inclusion = np.all(np.logical_and(latticefcc>=.0,  latticefcc <=1.), axis=1)
latticefcc = np.vstack((zlattice, latticefcc[inclusion,:]))
pplot(latticefcc, 3)
# lattice.shape